the ipynb is refer: 
Create the premodel by ResNet50,Xception, InceptionV3, VGG16, VGG19

In [ ]:
write_pretrain_data = False

def write_gap(MODEL, image_size):
    #base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    base_model = MODEL(weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    print(MODEL.__name__)
    gen = ImageDataGenerator()
    batch_size = 16
    train_generator = gen.flow_from_directory("data0/train", image_size, shuffle=False, batch_size=batch_size)
    test_generator = gen.flow_from_directory("data0/test", image_size, shuffle=False, batch_size=batch_size, class_mode=None)
    #train = model.predict_generator(train_generator, train_generator.nb_sample)
    #test = model.predict_generator(test_generator,  test_generator.nb_sample )
    print("predict_generator train {}".format(math.ceil(train_generator.samples/batch_size)))
    train = model.predict_generator(train_generator, math.ceil(train_generator.samples/batch_size))
    print("train: {}".format(train.shape))
    print("predict_generator test {}".format(math.ceil(train_generator.samples/batch_size)))
    test = model.predict_generator(test_generator, math.ceil(test_generator.samples/batch_size))
    print("test: {}".format(test.shape))
    print("label: {}".format(train_generator.classes.shape))

    print("begin create database {}".format(Model.__name__))
    with h5py.File("models/premodel/gap_%s.h5" % MODEL.__name__) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)
    print("write_gap {} successed".format(Model.__name__))

if write_pretrain_data:
    write_gap(ResNet50, (224, 224))
    write_gap(Xception, (299, 299))
    write_gap(InceptionV3, (299, 299))
    write_gap(VGG16, (224, 224))
    write_gap(VGG19, (224, 224))

In [1]:
import h5py
import numpy as np
from sklearn.utils import shuffle

np.random.seed(2017)

def load_data(pretrain_data_file):
    print('load data: ' + pretrain_data_file)
    with h5py.File("models/premodel/" + pretrain_data_file, 'r') as h:
        X_train = np.array(h['train'])
        X_test = np.array(h['test'])
        y_train = np.array(h['label'])
        X_train, y_train = shuffle(X_train, y_train)
        return X_train, y_train, X_test
    

from keras.models import *
from keras.layers import *

def make_model(input_shape):
    input_tensor = Input(input_shape)
    x = Dropout(0.5)(input_tensor)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(input_tensor, x)

    model.compile(optimizer='adadelta',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model
print("Done")

Using TensorFlow backend.


Done


ResNet50

In [28]:
X_train, y_train, X_test = load_data("gap_ResNet50.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2)

load data: gap_ResNet50.h5
Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 0s - loss: 0.2903 - acc: 0.8735 - val_loss: 0.1170 - val_acc: 0.9620
Epoch 2/8
20000/20000 [==============================] - ETA: 0s - loss: 0.1262 - acc: 0.9526

Xception

In [2]:
X_train, y_train, X_test = load_data("gap_Xception.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2)

load data: gap_Xception.h5
Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 0s - loss: 0.2973 - acc: 0.9350 - val_loss: 0.1293 - val_acc: 0.9888
Epoch 2/8
20000/20000 [==============================] - 0s - loss: 0.0916 - acc: 0.9885 - val_loss: 0.0618 - val_acc: 0.9910

InceptionV3

In [2]:
X_train, y_train, X_test = load_data("gap_InceptionV3.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2)

load data: gap_InceptionV3.h5
Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 0s - loss: 0.2173 - acc: 0.9453 - val_loss: 0.0844 - val_acc: 0.9868
Epoch 2/8
20000/20000 [==============================] - 0s - loss: 0.0700 - acc: 0.9870 - val_loss: 0.0470 - val_acc: 0.9896

VGG16

In [32]:
X_train, y_train, X_test = load_data("gap_VGG16.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2)

load data: gap_VGG16.h5
Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 0s - loss: 2.2382 - acc: 0.6573 - val_loss: 0.3411 - val_acc: 0.8940
Epoch 2/8
20000/20000 [==============================] - 0s - loss: 0.8535 - acc: 0.8368 - val_loss: 0.2249 - val_acc: 0.9346

VGG19

In [33]:
X_train, y_train, X_test = load_data("gap_VGG19.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2)

load data: gap_VGG19.h5
Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 0s - loss: 1.8230 - acc: 0.7036 - val_loss: 0.3164 - val_acc: 0.9086
Epoch 2/8
20000/20000 [==============================] - 0s - loss: 0.7688 - acc: 0.8470 - val_loss: 0.1945 - val_acc: 0.9406

combine  ResNet50,Xception and InceptionV3

In [51]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:
    print('------------------'+filename)
    with h5py.File("models/premodel/" + filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)
print(X_train.shape)
print(y_train.shape)

------------------gap_ResNet50.h5
------------------gap_Xception.h5
------------------gap_InceptionV3.h5
(25000, 6144)
(25000,)


In [49]:
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 0s - loss: 0.4674 - acc: 0.8471 - val_loss: 0.0944 - val_acc: 0.9658
Epoch 2/8
20000/20000 [==============================] - 0s - loss: 0.1606 - acc: 0.9438 - val_loss: 0.0770 

In [54]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in ["gap_ResNet50.h5",  "gap_VGG16.h5", "gap_VGG19.h5"]:
    print('------------------'+filename)
    with h5py.File("models/premodel/" + filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)
print(X_train.shape)
print(y_train.shape)

------------------gap_ResNet50.h5
------------------gap_VGG16.h5
------------------gap_VGG19.h5
(25000, 3072)
(25000,)


In [55]:
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 0s - loss: 0.5456 - acc: 0.8558 - val_loss: 0.1035 - val_acc: 0.9640
Epoch 2/8
20000/20000 [==============================] - 0s - loss: 0.2034 - acc: 0.9425 - val_loss: 0.0884 

In [56]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("data0/test", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred.csv', index=None)
df.head(10)

10720/12500 [========================>.....] - ETA: 0s Found 12500 images belonging to 1 classes.


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.995
4,5,0.005
5,6,0.005
6,7,0.005
7,8,0.005
8,9,0.005
9,10,0.005
